# 2-step 1 qubit DME 
Reproducing DME proposal Figure 5: involving 10 qubits, 2 steps of DME and each step with trotter iterations $M=3$.

We define in this example $\sigma=|0\rang\lang 0|$ and $\rho=|+\rang\lang+|$.

The first step we use DME $(\hat E_{t/M}^{\rho}(\sigma))^M$ to get $e^{-it\rho}\sigma e^{it\rho}$.
The second step is a recursion, letting $\rho' = e^{-it\rho}$, we get $e^{-it\rho'}\sigma e^{it\rho'}$

In [1]:
# imports
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
import numpy as np
import scipy
from utils import *
from copy import deepcopy
from qiskit_experiments.library import StateTomography
from qiskit_aer.primitives import SamplerV2
from qiskit.quantum_info import Statevector, DensityMatrix
from qiskit_ibm_runtime.fake_provider import *

/Users/pethidine/Documents/GitHub/Qibo-IonQ/DME/utils.py:170: SyntaxWarning: invalid escape sequence '\s'
  '''Tr(\sqrt{\sqrt dm_1)dm_2\sqrt{\sqrt dm_1)})^2'''


In [2]:
# create reference state
import scipy.linalg
t = np.pi/3
state_0 = np.outer(np.array([1,0]),np.array([1,0]))
state_p = np.outer(np.array([1,1]),np.array([1,1]))/2
sigma = state_0
rho = state_p
sigma_step1 = scipy.linalg.expm(-1j*t*rho) @ sigma @ scipy.linalg.expm(1j*t*rho)
sigma_step2 = scipy.linalg.expm(-1j*t*sigma_step1) @ sigma @ scipy.linalg.expm(1j*t*sigma_step1)
print(sigma_step1)
print(sigma_step2)

[[ 7.50000000e-01-2.77555756e-17j -2.17265947e-17+4.33012702e-01j]
 [ 2.31043892e-18-4.33012702e-01j  2.50000000e-01+1.38777878e-17j]]
[[ 0.8125+2.72216170e-18j -0.375 +1.08253175e-01j]
 [-0.375 -1.08253175e-01j  0.1875+8.79816492e-18j]]


We label the 13 qubits:

.0  ./  ./   ./

.1  .2   .3     .4

.5  .6  .7  .8

.9  .10  .11     .12

Step 1, we have $\sigma$ state at [1, 8, 10, 3]. Correspondingly, $\rho$ state [0,2,5], [9,6,10], [4,7,12]. In this step, qubit 3 is not modified.

Step 2, swap between [1,2], [6,10], [7,8].

Step 3, DME on qubit 3 with the new memory qubits at [2, 7].

Step 4, swap between [6,2].

Step 5, DME on qubit 3 with memory qubit 2.


In [3]:
def trotter_DME(qc, target_id, instruction_id, t):
    '''Run DME on target qubit with $M\times$ instruction_qubit, each rotates with time $t/M$
    instruction_id: list
    '''
    M = len(instruction_id)
    s = t/M
    for rho_id in instruction_id:
        add_partial_swap(qc, target_id, rho_id, s)

In [4]:
# create circuit
qc = QuantumCircuit(13,2)
t = np.pi/4
# prepare initial state
# plus state on qubits
for i in range(13):
    if i not in [1,8,10,3]:
        qc.h(i)

# Step1: 3 trotter steps
step1_target = [1, 8, 10]
step1_instruction = [[0,2,5],[9,6,10],[4,7,12]]
for i, target_id in enumerate(step1_target):
    instruction_id = step1_instruction[i]
    trotter_DME(qc, target_id, instruction_id, t)

In [5]:
# Test fidelity of step 1
from qiskit_ibm_runtime.fake_provider import *
fakebackend = FakeOsaka()
tomo_backend = AerSimulator.from_backend(fakebackend)
physical_qubits = [i for i in range(13)]

dm_sim = qiskit_tomography(qc, tomo_backend, 1000, physical_qubits,[1,8,10])
print(dm_sim)
# state_vec = Statevector.from_instruction(qc)
# dm_expect = DensityMatrix(state_vec).data
# print(fidelity(dm_sim, dm_expect))